In [1]:
#!pip install simpletransformers
from config import *
import pandas as pd
from utils import *
import logging
from sklearn.model_selection import train_test_split 
from simpletransformers.classification import (
    MultiLabelClassificationModel, MultiLabelClassificationArgs
)



logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [44]:
data_dir=list_dir['data_dir']
df = pd.read_csv(data_dir,encoding='utf-8')


input_data=df['Title_description']
targets =df[label_['label_list']]

In [45]:
input_data[45]

'৯ কোম্পানি জানালো পর্ষদ সভার তারিখ: শেয়ারবাজারে তালিকাভুক্ত ৯টি কোম্পানির পরিচালনা পর্ষদ তাদের প্রান্তিক প্রতিবেদন প্রকাশ সংক্রান্ত পরিচালনা পর্ষদ সভার তারিখ ঘোণষা করেছে।বুধবার (১৯ জানুয়ারি) ঢাকা স্টক এক্সচেঞ্জ (ডিএসই) সূত্রে এ তথ্য জানা গেছে। কোম্পানিগুলো হলো- আরএন স্পিনিং, দেশবন্ধু পলিমার, তমিজউদ্দিন টেক্সটাইল, ফার কেমিক্যাল, এমএল ডাইং, মুন্নু সিরামিক, মুন্নু ফেব্রিক্স, মুন্নু এগ্রো ও রানার অটোমোবাইল।কোম্পানিগুলোর মধ্যে আরএন স্পিনিংয়ের বোর্ড সভা ২৪ জানুয়ারি বিকাল ৩টায়, দেশবন্ধু পলিমারের ২৪ জানুয়ারি বিকাল সাড়ে ৩টায়, তমিজউদ্দিন টেক্সটাইলের ২৫ জানুয়ারি বিকাল সাড়ে ৩টায়, ফার কেমিক্যালের ২৪ জানুয়ারি বিকাল ৪টায়, এমএল ডাইংয়ের ২৪ জানুয়ারি বিকাল সাড়ে ৩টায়, মুন্নু সিরামিকের ২৫ জানুয়ারি সন্ধ্যা ৬টায়, মুন্নু ফেব্রিক্সের ২৫ জানুয়ারি বিকাল সাড়ে ৪টায়, মুন্নু এগ্রোর ২৫ জানুয়ারি বিকাল সাড়ে ৩টায় এবং রানার অটোমোবাইলের ২৪ জানুয়ারি বিকাল ৩টায় অনুষ্ঠিত হবে।কোম্পানিগুলোর বোর্ড সভায় ৩১ ডিসেম্বর ২০২১ সমাপ্ত ছয় মাসের অনিরীক্ষিত আর্থিক প্রতিবেদন পর্যালোচনা করে তা শেয়ারহোল্ডারদের জন্য প্রকাশ করা হবে।'

In [46]:
# set up ppreprocessing pipeline 
def remove_stopwords(text):
    text = ' '.join([word for word in text.split() if word not in pre_process_config['stop_word']])
    return text

input_data = input_data.apply(lemmatize_text)
input_data= input_data.apply(remove_prone)
input_data = input_data.apply(remove_numbers)
input_data = input_data.apply(remove_english_words)
input_data = input_data.apply(remove_stopwords)
input_data = input_data.apply(remove_perenthesis)
input_data = input_data.apply(remove_dash)
input_data = input_data.apply(remove_extra_spaces)
input_data = input_data.apply(remove_3rdbreket)
#input_data = input_data.apply(remove_symbols) 


In [47]:
input_data[45]

'কোম্পানি জানালো পর্ষদ সভার তারিখ: শেয়ারবাজারে তালিকাভুক্ত কোম্পানির পরিচালনা পর্ষদ প্রান্তিক প্রতিবেদন প্রকাশ সংক্রান্ত পরিচালনা পর্ষদ সভার তারিখ ঘোণষা করেছে।বুধবার ঢাকা স্টক এক্সচেঞ্জ সূত্রে তথ্য গেছে। কোম্পানিগুলো হলো- আরএন স্পিনিং, দেশবন্ধু পলিমার, তমিজউদ্দিন টেক্সটাইল, ফার কেমিক্যাল, এমএল ডাইং, মুন্নু সিরামিক, মুন্নু ফেব্রিক্স, মুন্নু এগ্রো রানার অটোমোবাইল।কোম্পানিগুলোর আরএন স্পিনিংয়ের বোর্ড সভা জানুয়ারি বিকাল টায়, দেশবন্ধু পলিমারের জানুয়ারি বিকাল সাড়ে টায়, তমিজউদ্দিন টেক্সটাইলের জানুয়ারি বিকাল সাড়ে টায়, ফার কেমিক্যালের জানুয়ারি বিকাল টায়, এমএল ডাইংয়ের জানুয়ারি বিকাল সাড়ে টায়, মুন্নু সিরামিকের জানুয়ারি সন্ধ্যা টায়, মুন্নু ফেব্রিক্সের জানুয়ারি বিকাল সাড়ে টায়, মুন্নু এগ্রোর জানুয়ারি বিকাল সাড়ে টায় রানার অটোমোবাইলের জানুয়ারি বিকাল টায় অনুষ্ঠিত হবে।কোম্পানিগুলোর বোর্ড সভায় ডিসেম্বর সমাপ্ত ছয় মাসের অনিরীক্ষিত আর্থিক প্রতিবেদন পর্যালোচনা শেয়ারহোল্ডারদের প্রকাশ হবে।'

In [5]:
#print 10 rows of targets with loc 
col_list=bert_label_maker(targets) 
#input_data=preprocess_news(input_data)
df['lebel']=col_list 

new_df=pd.DataFrame({'Title_description':input_data,'lebel':col_list})
new_df=new_df.dropna()
new_df=new_df.reset_index(drop=True)
new_df.head(10)

,Title_description,lebel
0,মস উৎপদন বনধ বড ওয়লড অসততব সকট শয়রবজরর তলকভকত ...,"[0, 0, 0, 0, 0, 0, 0, 0, 1]"
1,এসইএমএল গরথ ফনডর চদ গরহণ চলছ এসইএমএল এফবএলএসএল...,"[0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,জএসপ ফইননসর লভযশ ঘষণ বযক বহরভত আরথক খতর কমপন জ...,"[0, 0, 0, 0, 0, 0, 0, 0, 1]"
3,যমন অয়লর শয়রপরত আয় বযপক বড়ছ শয়রবজর তলকভকত জবলন...,"[0, 1, 1, 0, 0, 0, 0, 0, 0]"
4,কমপনর আয়বযয়র পরকশ পজবজর তলকভকত কমপনর পরচলন পরষ...,"[0, 0, 1, 0, 0, 0, 0, 0, 0]"
5,কমপনর আরথক পরতবদন পরকশ দবতয় পরনতকর অনরকষত আরথক...,"[0, 0, 1, 0, 0, 0, 0, 0, 0]"
6,কষতগরসত কটর ময়দ বড়ল বছর শয়রবজনস পরতবদক কষতগরস...,"[0, 0, 0, 0, 0, 0, 0, 0, 0]"
7,জকউ বলপনর শয়রদর বড়ছ ঢক সটক একসচঞজ গতকল জকউ বল...,"[1, 0, 1, 0, 0, 0, 0, 0, 0]"
8,মনফ কমছ ফওয় ফডর শয়রবজর তলকভকত ফওয় ফড লমটড চলত ...,"[0, 0, 1, 0, 0, 0, 0, 0, 0]"
9,ভল লভযশর গজব রপবলক ইনসযরনসর দর মস দবগন বম খতর ...,"[0, 0, 0, 0, 0, 0, 0, 0, 1]"


In [6]:

train, eval = train_test_split(new_df, test_size=split_config['test_size'], random_state=split_config['random_state'])   
train.shape

(8238, 2)

In [ ]:
model = MultiLabelClassificationModel(
     bert_model_config['model_type'], bert_model_config['model_name'],
    num_labels=num_labels,args=simple_transformer_config
  
)

Some weights of the model checkpoint at sagorsarker/bangla-bert-base were not used when initializing BertForMultiLabelSequenceClassification: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultiLabelSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultiLabelSequenceClassificatio

In [ ]:
model.train_model(train)

/usr/local/lib/python3.8/dist-packages/simpletransformers/classification/classification_model.py:612: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(
Saving vocabulary to /content/drive/MyDrive/news_data/checkpoint-1030-epoch-1/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to /content/drive/MyDrive/news_data/checkpoint-1030-epoch-1/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to /content/drive/MyDrive/news_data/checkpoint-1030-epoch-1/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to /content/drive/MyDrive/news_data/checkpoint-1030-epoch-1/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to /content/drive/MyDrive/news_dat

(2060, 0.07479254769577275)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(eval)

/usr/local/lib/python3.8/dist-packages/simpletransformers/classification/classification_model.py:1454: UserWarning: Dataframe headers not specified. Falling back to using column 0 as text and column 1 as labels.
  warnings.warn(


In [ ]:
print(result)
print(model_outputs)


{'LRAP': 0.978943304877883, 'eval_loss': 0.05170597892980209}
[[2.85530090e-03 9.96093750e-01 9.96093750e-01 ... 3.27301025e-03
  1.73377991e-03 4.38308716e-03]
 [6.71863556e-04 6.31332397e-04 1.09481812e-03 ... 3.24964523e-04
  5.48362732e-04 1.74140930e-03]
 [1.15585327e-03 1.83105469e-03 4.66537476e-03 ... 5.02014160e-03
  1.23023987e-03 9.96093750e-01]
 ...
 [9.44824219e-01 8.06427002e-03 9.97558594e-01 ... 3.55148315e-03
  5.34439087e-03 7.54928589e-03]
 [9.85839844e-01 4.77218628e-03 6.46209717e-03 ... 4.23049927e-03
  3.82423401e-03 4.48608398e-03]
 [4.65631485e-04 9.98535156e-01 1.11579895e-03 ... 6.43730164e-04
  4.10795212e-04 7.88688660e-04]]


In [ ]:
news="রেইসের ফান্ডের লভ্যাংশ ঘোষণা: পুঁজিবাজারের তালিকাভুক্ত মিউচ্যুয়াল ফান্ডের ইউনিট হোল্ডারদের লভ্যাংশ ঘোষণা ট্রাস্টি। গত রোববার ফান্ডগুলোর ট্রাস্টি সভায় জুন সমাপ্ত হিসাব বছরের লভ্যাংশ ঘোষণা হয়। ফান্ডের লভ্যাংশের রেকর্ড ডেট ঘোষণা সেপ্টেম্বর।এবি ব্যাংক ফার্স্ট মিউচ্যুয়াল, ইবিএল এনআরবি মিউচ্যুয়াল ফান্ড, পপুলার লাইফ ফার্স্ট মিউচ্যুয়াল ফান্ড, পিএইচপি ফার্স্ট মিউচ্যুয়াল ফান্ড, ইবিএল ফার্স্ট মিউচ্যুয়াল ফান্ড, ট্রাস্ট ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ড, আইএফআইসি ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ড, এক্সিম ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ড, ফার্স্ট বাংলাদেশ ফিক্সড ইনকাম ফান্ড ফার্স্ট জনতা ব্যাংক মিউচ্যুয়াল ফান্ডের ইউনিট হোল্ডারদের লভ্যাংশ ঘোষণা হয়েছে। ফান্ডগুলোর ফান্ডের ইউনিট হোল্ডারদের শতাংশ, তিনটির শতাংশ, দুটির শতাংশ চারটি ফান্ডের ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা ট্রাস্টি। পুরোটাই রি-ইনভেস্টমেন্ট ইউনিট।তথ্য মতে, এবি ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।ইবিএল এনআরবি মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।পপুলার লাইফ ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।পিএইচপি ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।ইবিএল ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।ট্রাস্ট ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।আইএফআইসি ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।এক্সিম ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা।ফার্স্ট বাংলাদেশ ফিক্সড ইনকাম ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।ফার্স্ট জনতা ব্যাংক মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।"

#preprocess news
news= preprocess_news(news)

#preprocess news with pipeline 



In [ ]:
predictions, raw_outputs = model.predict(["রেইসের ফান্ডের লভ্যাংশ ঘোষণা: পুঁজিবাজারের তালিকাভুক্ত মিউচ্যুয়াল ফান্ডের ইউনিট হোল্ডারদের লভ্যাংশ ঘোষণা ট্রাস্টি। গত রোববার ফান্ডগুলোর ট্রাস্টি সভায় জুন সমাপ্ত হিসাব বছরের লভ্যাংশ ঘোষণা হয়। ফান্ডের লভ্যাংশের রেকর্ড ডেট ঘোষণা সেপ্টেম্বর।এবি ব্যাংক ফার্স্ট মিউচ্যুয়াল, ইবিএল এনআরবি মিউচ্যুয়াল ফান্ড, পপুলার লাইফ ফার্স্ট মিউচ্যুয়াল ফান্ড, পিএইচপি ফার্স্ট মিউচ্যুয়াল ফান্ড, ইবিএল ফার্স্ট মিউচ্যুয়াল ফান্ড, ট্রাস্ট ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ড, আইএফআইসি ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ড, এক্সিম ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ড, ফার্স্ট বাংলাদেশ ফিক্সড ইনকাম ফান্ড ফার্স্ট জনতা ব্যাংক মিউচ্যুয়াল ফান্ডের ইউনিট হোল্ডারদের লভ্যাংশ ঘোষণা হয়েছে। ফান্ডগুলোর ফান্ডের ইউনিট হোল্ডারদের শতাংশ, তিনটির শতাংশ, দুটির শতাংশ চারটি ফান্ডের ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা ট্রাস্টি। পুরোটাই রি-ইনভেস্টমেন্ট ইউনিট।তথ্য মতে, এবি ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।ইবিএল এনআরবি মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।পপুলার লাইফ ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।পিএইচপি ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।ইবিএল ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।ট্রাস্ট ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।আইএফআইসি ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।এক্সিম ব্যাংক ফার্স্ট মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা।ফার্স্ট বাংলাদেশ ফিক্সড ইনকাম ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।ফার্স্ট জনতা ব্যাংক মিউচ্যুয়াল ফান্ডের ট্রাস্টি ইউনিট হোল্ডারদের শতাংশ লভ্যাংশ ঘোষণা করেছে। আলোচিত সময়ে ফান্ডটির ইউনিট আয় পয়সা। ইউনিট কস্ট প্রাইস টাকা পয়সা। বাজারমূল্য ফান্ডটির এনএভি টাকা পয়সা।"])

In [ ]:
predictions

[[0, 0, 0, 0, 0, 0, 0, 0, 1]]